# Learn Sparse Arrays by Building a Movie Recommender


We will build a movie recommendation system to learn scipy's sparse array handling. We'll use data from the [GroupLens](https://grouplens.org) project, an effort by the University of Minnesota Department of Computer Science.  Among other things, GroupLens researches recommender systems.

In [0]:
#requisite libraries:
import numpy as np
import pandas as pd
from urllib.request import urlopen


## Data Files
The data for this exercise comes from publicly available datasets maintained and provided by GroupLens.  The data files are available [here](http://grouplens.org/datasets/).  Please see the [Readme](http://files.grouplens.org/datasets/movielens/ml-latest-README.html) for a description of the complete dataset.

For our purposes, we need only two files from the complete dataset.  These two files files, nearly 1 GB in size have been downloaded, read into pandas dataframes, pickled and uploaded to AWS S3 storage so that they are available for easy access.

**These files may disappear. If you intend to continue using this data after tonights session, please get your own copy from the source.**

For this exercise, we need the movies.csv file and the ratings.csv file.

### Movies

In [0]:
# read pickled data files
path = 'https://focods.s3.us-east-2.amazonaws.com/movies.pcl'
movies = pd.read_pickle(urlopen(path), compression=None)


In [0]:
#@title
movies.head()

In [0]:
len(movies)

### Ratings

In [0]:
#Ratings: (takes about 20 seconds)
path = 'https://focods.s3.us-east-2.amazonaws.com/ratings.pcl'
ratings = pd.read_pickle(urlopen(path), compression=None)


In [0]:
ratings.head()

In [0]:
len(ratings)

From the [Readme](http://files.grouplens.org/datasets/movielens/ml-latest-README.html), the definition of the  rating column is:
> >Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

For our purposes let's just look at the positive ratings, i.e., those ratings greater than 3.0. Our recommender system will essentially answer
*if you liked Movie X, you might like Movie Y*

In [0]:
# get just the good ratings
good_ratings = ratings.query('rating > 3.0')

In [0]:
len(good_ratings)

## The Co-Occurrence Matrix

The heart of this recommender system is a co-occurrence matrix. This matrix has dimensions *movies x movies*. Cell $c_{ij}$ captures the number of users who liked movie $i$ **and** movie $j$.  Essentially the recommender will, for a given argument movie, find the other movies that users also liked and return the movies that were most liked.

### Make Users x Movies Sparse Array

We'll calculate the cooccurance matrix from an intermediate matrix that shows which users liked which movies. This matrix conceptually has one row for each user and one column for each movie. We can use the `UserId` and `MovieId` columns in the `good_ratings` data frame to index this matrix which we'll call `user_ratings`. Cell $ur_{ij}$ in `user_ratings` will contain the value $1$ if user $i$ liked movie $j$ otherwise 0.

How big will the `user_ratings` matrix be?


In [0]:
n_users = good_ratings.userId.max() +1 # to count for index 0
n_movies = good_ratings.movieId.max() +1 # to count for index 0

print (f'Max userId: {n_users}')
print (f'Max movieId: {n_movies}')
print(f'Size of users x movies: {n_users*n_movies}')

55 billion cells? *Yikes!* Sparse Arrays to the rescue!!

Most users like a tiny subset of the total number of movies which implies that most of the cells in `user_ratings` will be zero. A matrix in which most of its values are 0 is called a *sparse matrix*.   We can drastically reduce the memory requirements of a matrix by storing it in sparse form. THis means storing just the non-zero values and some means of determining the row and column coordinates for the non-zero values.

`scipy.sparse` provides a set of classes, methods and utility functions for manipulating sparse matrices. See [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html).

There are seven types of sparse matrices in implemented `scipy.sparse`.  Our recommender system will be doing manipulations by row, so we'll select the  compressed sparse row or [csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) class.

In [0]:
from scipy.sparse import csr_matrix

We will create the user ratings matrix using the csr_function below. Note that we are following the syntax in the second example shown on the [csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) page. In that example, the first argument to `csr_matrix` is a tuple: `(d, (r, c))` where d is the data vector, r is a vector of row indices and c is a vector of column indices. Vectors d, r and c must all be of the same length. This produces $csr[r[i],c[i]] = d[i$] and implicitly 0 everywhere else.

In our case, we just need the value of  $1$ to indicate that user $i$ liked movie $j$ and we need as many $1$'s as we have good ratings.

In [0]:
user_ratings = csr_matrix(   ([1]*len(good_ratings), #data vector
                              (good_ratings.userId, good_ratings.movieId)), # row and column vectors, resp.
                          dtype=np.int32) #element data type


In [0]:
user_ratings

## Calculate the Co-occurrence Matrix
The dimensions of `user_ratings` (shorthand $U$) is *users x movies*,  its transpose is of dimension *movies x users*, then $U^TU$ is of dimension *movies x  movies*. Let:
$$
C = U^TU
$$
Let's look at cell $C_{i,j}$. From the definition of matrix multiplication,
$$
C_{i,j} = \sum{U^T_{i,\centerdot}*U_{\centerdot, j}}
$$

In other words, $C_{i,j}$ is the sum of the product of the $i^{th}$ row of $U$'s transpose and the $j^{th}$ column of $U$.

Note that $U^T_{i,\centerdot}$ is just $U_{\centerdot,i}$, so term being summed above becomes $U_{\centerdot,i}*U_{\centerdot, j}$ which is element-wise product of the $i^{th}$ column of $U$ and the $j^{th}$ column of $U$.
The $i^{th}$ column of $U$ indicates whether a user liked movie $i$ and the $j^{th}$ column of $U$ indicates whether a user liked movie $j$. Multiplying the two columns together produces a  $1$ if a user liked _both_ movies $i$ and $j$. Add up the users who liked  both movies $i$ and $j$ to get a count of the number of users who liked both movies.
So $C_{i,j}$ contains the count of the users that liked both movie $i$ and movie $j$.

Note that $C$ is symmetric: $C_{i,j} = C_{j,i}$, that is to say the number of users that liked movies $i$ and $j$ is the same as the number of users that liked movies $j$ and $i$.
Note further that $C_{i,i}$, the main diagonal of $C$, gives a count of the number of users that liked movie $i$.

Let's calculate that matrix!


In [0]:
# takes about 1 minute to run
co_occurrence = user_ratings.transpose().dot(user_ratings)

In [0]:
co_occurrence

The shape of the co_occurence matrix is movies x movies. **the cell co_occurrence[i,j] is the number of users who liked movie i AND movie j**

In [0]:
# will need the total number of likes for each movie later for Jaccard scoring
# total likes of each movie is main diagonal of co_occurrence matrix
tot_likes = co_occurrence.diagonal()
tot_likes.shape


In [0]:
type(tot_likes)

Notice that the result of `diagonal()` is a dense `numpy.ndarray`

Some house keeping is in order. $C_{i,i} \geq C_{i,j} \forall j$ which is to say that the number of times movie $i$ is liked is at least as large as the number of times movie $i$ paired with any movie $j$ is liked. The users that liked the pair $(i,j)$ is a subset of the users that liked movie $i$.  So recommendations for movie $i$ based pair-wise likes will always include movie $i$. A system that produces a recommendation of the same movie that you just liked is probably less than ideal. Imagine getting a recommendation to the effect "if you liked *The Phantom Menace* you might also like *The Phantom Menace*." Not very useful.

We can prevent this simply by zeroing out the main diagonal of the co-occurrence matrix.

In [0]:
# zero out the main diagonal, so that recommendations aren't self-referential
co_occurrence.setdiag(0)
# just to be tidy, eliminate any zeros introduced above
co_occurrence.eliminate_zeros()
co_occurrence

## Step-by-Step Recommender

### First, Some Helper Routines

We'll create some functions to make life easy and do index-to-string and string-to-index conversion

In [0]:
def get_movieId(moviename):
  """
  return the movie id given the title/moviename
  """
  mvindex = movies.query('title == @moviename').index
  if len(mvindex) == 0:
    raise Exception(f'Invalid movie name: {moviename}')
  return mvindex[0] # just the first one in case there were multiples
  

In [0]:
#run this cell to test invalid input
#get_movieId('jdkjkj')

In [0]:
get_movieId('Father of the Bride Part II (1995)')

In [0]:
def find_movies(title_regexp):
  """
  searches the list of movies with a reg exp
  """
  return movies.title[movies.title.str.match(title_regexp, case=False)]

In [0]:
find_movies('.*toy story.*')

### Build the Recommender Step-by-Step

In [0]:
#get the movieId for the movie for which we'll make recommendations
# fob = Father of the Bride
fob = get_movieId('Father of the Bride Part II (1995)')

# what other movies did the people who liked FOB like?
# this is the ROW in the co_occurrence matrix corresponding to the movieId
fob_cohorts = co_occurrence.getrow(fob)
fob_cohorts

In [0]:
# the movieIds of the cohorts
fob_cohort_ids = fob_cohorts.indices #i.e. the column indexes which are movieIds

# the number of times each cohort was liked
fob_cohort_vals = fob_cohorts.data

In [0]:
#get the top 5 cohort ids

#sort the number of times liked into ascending order
fob_cohort_sort = fob_cohort_vals.argsort()

# slicer for the last 5 elements of a vector in reverse order
slicer = slice(-1,-6,-1)


# grab the last 5 id's and times liked using the slicer
top5_ids  = fob_cohort_ids [fob_cohort_sort[slicer]]
top5_vals = fob_cohort_vals[fob_cohort_sort[slicer]]

In [0]:
top5_ids

In [0]:
top5_vals

In [0]:
# look up the titles assoc'd with the top 5 ids
movies.loc[top5_ids].title

So the people who liked *Father of the Bride* also liked *Toy Story*, *Forrest Gump* etc.

## Jaccard Normalization

Who doesn't like *Toy Story* or *Forrest Gump*?  Chances are decent that whatever else a user liked, that user also liked one of these two popular movies. So the algorithm above which sorts just on the number of users that liked a movie, will have a tendency to recommend the most popular movies regardless of the argument movie.

We can employ a normalization technique in our scoring that will reduce this tendency. The technique is called [Jaccard Normalization](https://en.wikipedia.org/wiki/Jaccard_index) and it essentially is an *intersection over union (IOU)* computation. We calculate the number of people who liked *both* movies A and B (the intersection) and divide that count by the number of people who liked *either* movies  A or B (the union). The higher the ratio, the higher the relevance of movie B to movie A.

The figure below illustrates Jaccard Normalization for two pairs of movies: *Father of the Bride* and *Toy Story* on the left side of the figure and *Father of the Bride* and *Grumpier Old Men* on the right. In the case of *Toy Story* a lot of people liked it, so the denominator of the Jaccard index will be much larger than the denominator in the case of *Grumpier*.  Even though the pair-like in both cases is roughly the same, the denominators of the two Jaccards are quite a bit different. Thus the Jaccard index helps to prevent the recommendation alogorithm from always recommending the most popular movies.

In [0]:
#@title

import matplotlib.pyplot as plt

%matplotlib inline
from matplotlib_venn import venn2

def get_subsets(m1, m2):
  mid1 = get_movieId(m1)
  mid2 = get_movieId(m2)
  tot_likes1 = tot_likes[mid1]
  tot_likes2 = tot_likes[mid2]

  tot_likesBoth = co_occurrence.getcol(mid2).toarray()[mid1][0]
  
  return (tot_likes1-tot_likesBoth, tot_likes2-tot_likesBoth, tot_likesBoth)

fig = plt.figure(figsize=(10,8))
fig.suptitle('Jaccard Normalization')
ax1 = plt.subplot(121)

p1 = ('Father of the Bride Part II (1995)','Toy Story (1995)')
s1 = get_subsets(*p1)
j1 = s1[2]/(s1[0]+s1[1]+s1[2])
venn2(ax=ax1, subsets = s1, set_labels=p1)
ax1.set_title(f'Jaccard = {j1:.2f}')

ax2 = plt.subplot(122)

p2 = ('Father of the Bride Part II (1995)','Grumpier Old Men (1995)')
s2 = get_subsets(*p2)
j2 = s2[2]/(s2[0]+s2[1]+s2[2])
venn2(ax=ax2, subsets = s2, set_labels=p2)
ax2.set_title(f'Jaccard = {j2:.2f}')

plt.show()


### Jaccard Step-by-Step

In this sectioin we'll follow the algorithm from the previous section,  but introduce Jaccard normalization.

In [0]:
#get the movieId for the movie for which we'll make recommendations
# fob = Father of the Bride
fob = get_movieId('Father of the Bride Part II (1995)')

# what other movies did the people who liked FOB like?
# this is the ROW in the co_occurrence matrix corresponding to the movieId
fob_cohorts = co_occurrence.getrow(fob)

In [0]:
# the movieIds of the cohorts
fob_cohort_ids = fob_cohorts.indices #i.e. the column indexes which are movieIds

# the number of times each cohort was liked
fob_cohort_vals = fob_cohorts.data

#### Here's the Jaccard Calculation

The Jaccard index for two movies $A$ and $B$ is:
$$
J(A,B) = \frac{|A \cap B|} {|A \cup B|} = \frac{|A \cap B|} {|A| + |B| - |A \cap B|}
$$

In [0]:
#calculate Jaccard scores from the number of likes
# Father of the Bride is Movie A, every other movie is a movie B
fob_likes = tot_likes[fob]

# number of times each cohort of Father of Bride was liked
fob_cohort_likes = tot_likes[fob_cohort_ids]

# now calculate the Jaccard
fob_jaccard = fob_cohort_vals/(fob_likes + fob_cohort_likes - fob_cohort_vals)



In [0]:
fob_likes, fob

In [0]:
#get the top 5 cohort ids

#sort the Jaccard values into ascending order
fob_cohort_sort = fob_jaccard.argsort()

# slicer for the last 5 elements of a vector in reverse order
slicer = slice(-1,-6,-1)


# grab the last 5 id's and Jaccard index using the slicer
top5_ids_jaccard  = fob_cohort_ids [fob_cohort_sort[slicer]]
top5_jaccard = fob_jaccard[fob_cohort_sort[slicer]]

In [0]:
top5_jaccard

In [0]:
# look up the titles assoc'd with the top 5 ids
movies.loc[top5_ids_jaccard].title

#### Results With and Without Normalization

In [0]:
pd.DataFrame( {'title_no_jaccard': movies.loc[top5_ids].title.values,
               'LikeCount': top5_vals,
               'title_with_jaccard': movies.loc[top5_ids_jaccard].title.values,
               'JaccardScore': top5_jaccard
              })

## Function to do it All

In [0]:
def get_recommendations(moviename, nrec=5, Jaccard=True):
  """
  Returns at most nrec movie recommenations given an input movie name
  using Jaccard normalization (default) or just number of co-occurrences
  """
  
  # which movie?
  movieId = get_movieId(moviename)
  
  #get the cohorts of this movie 
  #(number of users that liked each movie AND this movieId)
  cohorts = co_occurrence.getrow(movieId)
  
  #get the movieIds and # times liked for each cohort
  cohort_ids = cohorts.indices
  cohort_n   = cohorts.data
  
  if Jaccard:
    # normalize by Jaccard measure
    nlikes = tot_likes[movieId]
    cohort_jaccard = cohort_n/(nlikes+tot_likes[cohort_ids]-cohort_n)
    colname = 'Jaccard'
  else:
    # no normalization, just use the like counts
    cohort_jaccard = cohort_n
    colname = 'Score'
  
  #sort by Jaccard measure
  cohort_sort = cohort_jaccard.argsort() #note, argsort() only sorts ascending

  #slicer to get last nrec elements in reverse order
  last_n_slice = slice(-1, -1*(nrec+1), -1)
  
  #get the ids and jaccards of the last(highest) nrec elements
  last_n_ids =     cohort_ids[cohort_sort[last_n_slice]]
  last_n_jaccard = cohort_jaccard[cohort_sort[last_n_slice]]
  
  #create a dataframe to contain the results
  ret_df = pd.DataFrame({'title': movies.loc[last_n_ids].title,
                         colname:last_n_jaccard,
                         'movieId':last_n_ids}).set_index('movieId')
  
  #ship it!
  return ret_df


In [0]:
mname = 'Father of the Bride Part II (1995)'
get_recommendations(mname, nrec=10)

In [0]:
mname = 'Father of the Bride Part II (1995)'
get_recommendations(mname, nrec=10, Jaccard=False)

In [0]:
get_recommendations('Star Wars: Episode IV - A New Hope (1977)')

## Some Notable Recommendations

In [0]:
get_recommendations('The Shape of Water (2017)')

In [0]:
get_recommendations('Pulp Fiction (1994)')

In [0]:
find_movies('friday')

In [0]:
get_recommendations('Friday the 13th (2009)')

In [0]:
get_recommendations('Friday the 13th (2009)', Jaccard=False)

In [0]:
find_movies('Repo Man')

In [0]:
get_recommendations('Repo Man (1984)')

In [0]:
find_movies('.*owski.*')

In [0]:
get_recommendations('Big Lebowski, The (1998)')